# Задача 1. Обнаружение фальшивых новостей

**Фальшивые новости** — это ложная информация, распространяемая через социальные сети и другие сетевые СМИ для достижения политических или идеологических целей.

**Твоя  задача** -  используя библиотеку *sklearn* построить модель классического машинного обучения, которая может с высокой точностью более 90% определять, является ли новость реальной (***REAL***） или фальшивой（***FAKE***).

Ты должен самостоятельно изучить и применить к задаче <code>TfidfVectorizer</code> для извлечения признаков из текстовых данных и <code>PassiveAggressiveClassifier</code>.

Ты  можешь использовать [данный датасет](https://storage.yandexcloud.net/academy.ai/practica/fake_news.csv) для обучения.

- **Построй** матрицу ошибок (confusion matrix). 
- **Представь**, что ваш заказчик очень любит графики и диаграммы. 
- **Визуализируй** для него результаты там, где это возможно.